In [1]:
# generate structures using leruli
# choose a fancy basis set
# optimize with pbe0
# generate fragments
# calculate frag without optimization
# with optimization

# collect energies into one file
# energy energy_frag energy_ BDE BDE_relaxed

In [1]:
import ase.io as aio
from ase.units import Bohr
from ase import Atoms
import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

from pyscf import gto, dft, scf

# geometric
from pyscf.geomopt.geometric_solver import optimize

from ase.visualize import view

/home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
c = 'CC'
p = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_2/fragments_single/{c}_opt.xyz'
atoms = aio.read(p)
view(atoms)

<Popen: returncode: None args: ['/home/sahre/opt/miniconda3/envs/qm-tools/bi...>

## Look at structures

In [22]:
comps = ['[Li]C', '[Li]N','[Li]O', '[Li]F']
#comps = ['[BeH]C', '[BeH]N','[BeH]O', '[BeH]F']

for c in comps:
    #c = 'FF'
    p = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_2/diatomics_single/{c}_opt.xyz'
    atoms = aio.read(p)
    p2 = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_2/diatomics_single/{c}.xyz'
    atoms2 = aio.read(p2)
    #print(atoms.get_distance(0,1)-atoms2.get_distance(0,1))
    #view(atoms, viewer='ngl')
    view(atoms)

1.962869066225505
1.7233025736609238
1.5707648699439576
1.55986243


In [34]:
atoms.get_angle(a1=1, a2 = 0, a3=2)

122.10555798801741

In [6]:
atoms.get_distance(0,1)

1.5205183326747493

In [70]:
c = '[GaH2][GaH2]'

#c = '[BH2][BH2]'

p = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_4/fragments_single/{c}_opt.xyz'
atoms = aio.read(p)
view(atoms, viewer='ngl')

In [35]:
c = 'BB'
p = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_2/fragments_single/{c}_opt.xyz'
atoms = aio.read(p)
view(atoms, viewer='ngl')

In [36]:
atoms.get_angle(a1=1, a2 = 0, a3=2)

128.97151216296135

In [37]:
atoms.get_distance(0,1)

1.1915088648624514

### Fragments

In [ ]:
# for c in ['CC', 'NN', 'OO', 'FF']:
#     #c = 'FF'
#     p = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/fragments/{c}_opt.xyz'
#     atoms = aio.read(p)
#     p2 = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/fragments/{c}.xyz'
#     atoms2 = aio.read(p2)
#     print(atoms.get_distance(0,1)-atoms2.get_distance(0,1))
c = 'FF' 
p = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/fragments/{c}_opt.xyz'
# p = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/fragments/{c}.xyz'

atoms = aio.read(p)
#print(atoms.get_distance(0,1))
view(atoms, viewer='x3d')

# Prepare calculations

In [ ]:
# comps = ['CC','CN','CO','CF','NN','NO','NF','OO', 'OF', 'FF']
c = 'CC' #sys.argv[1]
atoms = aio.read(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_single/{c}.xyz')

pyscf_atoms = uqm.ase2pyscf(atoms)
basis = 'def2-tzvp'
mol = gto.Mole()
mol.atom = pyscf_atoms
mol.basis = basis
mol.build()

mf = dft.RKS(mol)
mf.xc = 'pbe0'
mol_eq = optimize(mf, maxsteps=100)

mf = dft.RKS(mol)
mf.kernel()

new_pos = mol_eq.atom_coords()*Bohr

atoms.set_positions(new_pos)
aio.write(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_single/{c}_opt.xyz', atoms)

with open(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_single/{c}_energy', 'w') as f:
    f.write(str(mf.e_tot)+'\n')

# fragment calculations

In [ ]:
comps = ['CC','NN', 'OO', 'FF']
for smiles in comps:
    structure = aio.read(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_single/{smiles}_opt.xyz')
    if structure.get_chemical_symbols()[0] == 'C':
#         structure.set_distance(1,0, dCO, indices=[0, 2, 3, 4], fix = 0)
        indices=[0, 2, 3, 4]
    elif structure.get_chemical_symbols()[0] == 'N':
#         structure.set_distance(0,1, dCO, indices=[0, 2, 3])
        indices=[0, 2, 3]
    elif structure.get_chemical_symbols()[0] == 'O':
#         structure.set_distance(0,1, dCO, indices=[0, 2])
        indices=[0, 2]
    elif structure.get_chemical_symbols()[0] == 'F':
#         structure.set_distance(0,1, dCO)
        indices=[0]
        
    frag = Atoms(numbers = structure.get_atomic_numbers()[indices], positions=structure.get_positions()[indices])
    aio.write(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/fragments/{smiles}.xyz', frag)


# collect data

In [ ]:
import glob
from matplotlib import pyplot as plt
import numpy as np
import ase.io as aio
from pyscf import gto

import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

In [ ]:
# total energy of diatomics
energies_diatomics = dict()
efiles = glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_single/*_energy')
for file in efiles:
    with open(file, 'r') as f:
        k = file.split('/')[-1].split('_')[0]
        energies_diatomics[k] = float(f.readline().strip('\n'))

# total energy of relaxed fragments
energies_frags = dict()
efiles = glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/fragments/*_energy_relaxed')
for file in efiles:
    with open(file, 'r') as f:
        k = file.split('/')[-1].split('_')[0][0]
        energies_frags[k] = float(f.readline().strip('\n'))

# bfe
bfe = dict()
for k in energies_diatomics.keys():
    bfe[k] = energies_diatomics[k] - (energies_frags[k[0]]+energies_frags[k[1]])

# nuc repulsion for fixed distance and only contribution of heteroatoms
structures = glob.glob("/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_single/*_opt.xyz")
e_nuc = dict()
from ase.units import Bohr
d = 1.421151094407886/Bohr

for s in structures:
    mol = aio.read(s)
    k = s.split('/')[-1].split('_')[0]
    
    fixed = mol.get_atomic_numbers()[0]*mol.get_atomic_numbers()[1]/d
    relaxed = mol.get_atomic_numbers()[0]*mol.get_atomic_numbers()[1]/(mol.get_distance(0, 1)/Bohr)
    e_nuc[k] = [fixed]
# bfe, bfe_el, bfe_nuc 
energies = dict()
for k in bfe.keys():
    bfe_el = bfe[k] - e_nuc[k][0]
    #bfe_elr = bfe[k] - e_nuc[k][1]
    
    energies[k] = [bfe[k], bfe_el, e_nuc[k][0]]
    
# save data
path = '/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/energies_dict'
uqm.save_obj(energies, path)

comps = ['CC','CN','CO','CF','NN','NO','NF','OO', 'OF', 'FF']
energies_ordered = []
for c in comps:
    energies_ordered.append(energies[c])
energies_ordered = np.array(energies_ordered)
np.savetxt('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/energies.txt', energies_ordered)

In [ ]:
energies

In [ ]:
energies_ordered

In [ ]:
comps = ['CC','CN','CO','CF','NN','NO','NF','OO', 'OF', 'FF']
bde_ordered = []
for c in comps:
    bde_ordered.append(-energies[c][0]*2600)

In [ ]:
ref_bde = np.array([359.2, 337.7, 370.2, 452, 259.6, 253.5, 283.6, 195.2, 194.9, 151.7])

plt.plot(comps, bde_ordered, '-o')
plt.plot(comps, ref_bde, '-o')

# Double bonds

In [ ]:
from ase.visualize import view
import ase.io as aio

In [ ]:
comps = ['dCC','dCN', 'dCO', 'dNN', 'dNO', 'dOO']
for c in comps:
    mol = aio.read(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_double/{c}_opt.xyz')
    view(mol)

In [ ]:
comps = ['dCC','dCN', 'dCO', 'dNN', 'dNO', 'dOO']
for c in comps:
    with open(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_double/{c}_energy') as f:
        print(c, f.readline().strip('\n'))

In [ ]:
# mean distance between heavy atoms
# needed for nuclear repulsion calculation

bond_type = 'double'

structures = glob.glob(f"/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_{bond_type}/*_opt.xyz")
from ase.units import Bohr
dist = []
for s in structures:
    mol = aio.read(s)
    dist.append(mol.get_distance(0, 1))
dist = np.array(dist)
mean_dist = dist.mean()


In [ ]:
bfe

In [ ]:
bond_type = 'double'
# total energy of diatomics
energies_diatomics = dict()
efiles = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_{bond_type}/*_energy')
for file in efiles:
    with open(file, 'r') as f:
        k = file.split('/')[-1].split('_')[0]
        energies_diatomics[k] = float(f.readline().strip('\n'))

# total energy of relaxed fragments
energies_frags = dict()
efiles = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/fragments_{bond_type}/*_energy_relaxed')
for file in efiles:
    with open(file, 'r') as f:
        k = file.split('/')[-1].split('_')[0][0:2]
        energies_frags[k] = float(f.readline().strip('\n'))

# bfe
bfe = dict()
for k in energies_diatomics.keys():
    bfe[k] = energies_diatomics[k] - (energies_frags[k[0:2]]+energies_frags['d'+k[2]])

# nuc repulsion for fixed distance and only contribution of heteroatoms
structures = glob.glob(f"/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/diatomics_{bond_type}/*_opt.xyz")
e_nuc = dict()
from ase.units import Bohr
d = mean_dist/Bohr

for s in structures:
    mol = aio.read(s)
    k = s.split('/')[-1].split('_')[0]
    
    fixed = mol.get_atomic_numbers()[0]*mol.get_atomic_numbers()[1]/d
    relaxed = mol.get_atomic_numbers()[0]*mol.get_atomic_numbers()[1]/(mol.get_distance(0, 1)/Bohr)
    e_nuc[k] = [fixed]
# bfe, bfe_el, bfe_nuc 
energies = dict()
for k in bfe.keys():
    bfe_el = bfe[k] - e_nuc[k][0]
    #bfe_elr = bfe[k] - e_nuc[k][1]
    
    energies[k] = [bfe[k], bfe_el, e_nuc[k][0]]
    
# save data
path = '/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/energies_double_dict'
uqm.save_obj(energies, path)

comps = ['dCC','dCN','dCO','dNN','dNO','dOO']
energies_ordered = []
for c in comps:
    energies_ordered.append(energies[c])
energies_ordered = np.array(energies_ordered)
np.savetxt('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/energies_double_singlet.txt', energies_ordered)

# third/fourth row

In [ ]:
import glob
import ase.io as aio
from ase.visualize import view
import numpy as np

import re
import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

In [ ]:
# elements = ['[SiH3]', 'P', 'S', 'Cl']

elements = ['[GeH3]', '[AsH2]', '[SeH]', 'Br']


smiles = []
for i in range(len(elements)):
    for j in range(i, len(elements)):
        smiles.append(elements[i]+elements[j])
with open('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_4/diatomics_single/smiles', 'w') as f:
    for e in smiles:
        f.write(e+'\n')

In [ ]:
paths = glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_4/diatomics_single/*.xyz')
for p in paths:
    view(aio.read(p))

### collect data

In [ ]:
# mean distance between heavy atoms
# needed for nuclear repulsion calculation

row = 'row_4'
system = 'diatomics'
bond_type = 'single'

structures = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/{system}_{bond_type}/*_d2.xyz')
from ase.units import Bohr
dist = []
for s in structures:
    mol = aio.read(s)
    dist.append(mol.get_distance(0, 1))
dist = np.array(dist)
mean_dist = dist.mean()
print(mean_dist)
print(dist.std())

In [ ]:
mean_dist

In [ ]:
energies_diatomics

In [ ]:
bfe

In [ ]:
row = 'row_3'
system = 'diatomics'
bond_type = 'single'

# if row == 'row_3':
#     frag_types = ['Si', 'P', 'S', 'Cl']
# elif row == 'row_4':
#     frag_types = ['[GeH3]', '[AsH2]', '[SeH]', 'Br']

# total energy of diatomics
energies_diatomics = dict()
efiles = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/{system}_{bond_type}/*d2_energy')
for file in efiles:
    with open(file, 'r') as f:
        k = file.split('/')[-1].split('_')[0]
        energies_diatomics[k] = float(f.readline().strip('\n'))

# total energy of relaxed fragments
# row = 'row_3'
system = 'fragments'
# bond_type = 'single'
                   
energies_frags = dict()
efiles = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/{system}_{bond_type}/*_energy_relaxed')
for file in efiles:
    with open(file, 'r') as f:
        smiles = file.split('/')[-1].split('_')[0]
        frags = uqm.fragmentize_smiles(smiles)
        frag_name = frags[0]
#         if len(re.findall(r"\[.*?\]", smiles)) > 0:
#             frag_name = re.findall(r"\[.*?\]", smiles)[0]
#         else:
#             frag_name = re.sub( r"([A-Z])", r" \1", smiles).split()[0]

        energies_frags[frag_name] = float(f.readline().strip('\n'))

# bfe
bfe = dict()
for smiles in energies_diatomics.keys():
#     if len(re.findall(r"\[.*?\]", smiles)) > 0:
#         frags = re.findall(r"\[.*?\]", smiles)
#     else:
#         frags = re.sub( r"([A-Z])", r" \1", smiles).split()
    frags = uqm.fragmentize_smiles(smiles)
    assert len(frags) == 2
    
    bfe[smiles] = energies_diatomics[smiles] - (energies_frags[frags[0]]+energies_frags[frags[1]])

# nuc repulsion for fixed distance and only contribution of heteroatoms
structures = glob.glob(f"/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/diatomics_{bond_type}/*_d2.xyz")
e_nuc = dict()
from ase.units import Bohr
d = mean_dist/Bohr

for s in structures:
    mol = aio.read(s)
    k = s.split('/')[-1].split('_')[0]
    
    fixed = mol.get_atomic_numbers()[0]*mol.get_atomic_numbers()[1]/d
    relaxed = mol.get_atomic_numbers()[0]*mol.get_atomic_numbers()[1]/(mol.get_distance(0, 1)/Bohr)
    e_nuc[k] = [fixed]
# bfe, bfe_el, bfe_nuc 
energies = dict()
for k in bfe.keys():
    bfe_el = bfe[k] - e_nuc[k][0]
    #bfe_elr = bfe[k] - e_nuc[k][1]
    
    energies[k] = [bfe[k], bfe_el, e_nuc[k][0]]
    
# save data
path = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/energies_{bond_type}_dict'
# uqm.save_obj(energies, path)

comps = ['[SiH3][SiH3]', '[SiH3]P', '[SiH3]S', '[SiH3]Cl', 'PP', 'PS', 'PCl', 'SS', 'SCl', 'ClCl']
# comps = ['[GeH3][GeH3]','[GeH3][AsH2]','[GeH3][SeH]','[GeH3]Br','[AsH2][AsH2]','[AsH2][SeH]','[AsH2]Br','[SeH][SeH]','[SeH]Br','BrBr']
energies_ordered = []
for c in comps:
    energies_ordered.append(energies[c])
energies_ordered = np.array(energies_ordered)
np.savetxt(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/energies_{bond_type}_d2.txt', energies_ordered)

In [ ]:
energies_ordered

In [ ]:
energies_ordered

In [ ]:
from matplotlib import pyplot as plt
plt.plot(energies_ordered[:,1], '-o')

In [ ]:
# frag_types = ['[GeH3]', '[AsH2]', '[SeH]', 'Br']
# comps = []
# for i in range(len(frag_types)):
#     for j in range(i, len(frag_types)):
#         comps.append(frag_types[i]+ frag_types[j])

In [ ]:
energies_ordered

# Mixed rows

In [ ]:
import glob
import ase.io as aio
from ase.visualize import view
import numpy as np

import re
import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

In [ ]:
frags_row_2 = ['C', 'N', 'O', 'F']
frags_row_3 = ['[SiH3]', '[PH2]', '[SH]', 'Cl']
frags_row_4 = ['[GeH3]', '[AsH2]', '[SeH]', 'Br']

row = 'mixed_3_4'
if row ==  'mixed_2_3':
    frags_row_i = frags_row_2
    frags_row_j = frags_row_3
elif row ==  'mixed_2_4':
    frags_row_i = frags_row_2
    frags_row_j = frags_row_4
elif row ==  'mixed_3_4':
    frags_row_i = frags_row_3
    frags_row_j = frags_row_4

smiles = []
for i in frags_row_i:
    for j in frags_row_j:
        smiles.append(i+j)
with open(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/smiles', 'w') as f:
    for e in smiles:
        f.write(e+'\n')

In [ ]:
row = 'mixed_2_3'

paths = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/*.xyz')
for p in paths:
    view(aio.read(p))

# same distance for all atoms

In [ ]:
# identify second group
# set distance second group


In [ ]:
import pysmiles as ps
import glob
import ase.io as aio
from ase.visualize import view

import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# row = 'row_4'
dist = 1.42

xyz_files_new = []

for row in ['row_2', 'row_3', 'row_4']:
    xyz_files = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/diatomics_single/*_opt.xyz')
    smiles = []
    for f in xyz_files:
        s = f.split('/')[-1].split('_')[0]
#         print(s)
        if 'Si' in s:
            s = s.replace('Si', '[SiH3]')
        smiles.append(s)


    for s, f in zip(smiles, xyz_files):
        atoms = aio.read(f)
        num_atoms = atoms.get_global_number_of_atoms()

        graph = ps.read_smiles(s)
        num_h1 = graph.nodes(data='hcount')[1]
        indices = [1] # second heavy atom
        indices.extend(list(range(num_atoms-num_h1, num_atoms))) # hydrogens attached to second heavy atom

        atoms_new = atoms.copy()
        atoms_new.set_distance(0, 1, dist, indices=indices, fix = 0)

        file_new = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/distance/{s}_d{str(dist)}.xyz'
        aio.write(file_new, images=atoms_new)
#         xyz_name.append(f'{s}_d{str(dist)}.xyz')
        xyz_files_new.append(file_new)

with open(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/distance/gnu_para_{dist}', 'w') as fs:
    for line in xyz_files_new:
        name = line.split('/')[-1]
        fs.write(f'python /home/sahre/git_repositories/APDFT/prototyping/atomic_energies/analysis/bonding_trends/singlepoint.py \'{name}\'\n')

In [ ]:
# total energy of relaxed fragments
energies_frags = dict()

for row in ['row_2', 'row_3', 'row_4']:
    system = 'fragments'
    bond_type = 'single'

    efiles = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/{system}_{bond_type}/*_energy_relaxed')
    for file in efiles:
        with open(file, 'r') as f:
            smiles = file.split('/')[-1].split('_')[0]
            frags = uqm.fragmentize_smiles(smiles)
            frag_name = frags[0]
    #         if len(re.findall(r"\[.*?\]", smiles)) > 0:
    #             frag_name = re.findall(r"\[.*?\]", smiles)[0]
    #         else:
    #             frag_name = re.sub( r"([A-Z])", r" \1", smiles).split()[0]

            energies_frags[frag_name] = float(f.readline().strip('\n'))

In [ ]:
# total energy of diatomics
energies_diatomics = dict()
efiles = glob.glob(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/distance/*{str(dist)}_energy')
for file in efiles:
    with open(file, 'r') as f:
        k = file.split('/')[-1].split('_')[0]
        energies_diatomics[k] = float(f.readline().strip('\n'))

# bfe
bfe = dict()
for smiles in energies_diatomics.keys():
    frags = uqm.fragmentize_smiles(smiles)
    assert len(frags) == 2
    
    bfe[smiles] = energies_diatomics[smiles] - (energies_frags[frags[0]]+energies_frags[frags[1]])

# nuc repulsion for fixed distance and only contribution of heteroatoms
structures = glob.glob(f"/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/distance/*_d{dist}.xyz")
e_nuc = dict()
from ase.units import Bohr
d = dist/Bohr

for s in structures:
    mol = aio.read(s)
    k = s.split('/')[-1].split('_')[0]
    
    fixed = mol.get_atomic_numbers()[0]*mol.get_atomic_numbers()[1]/d
    relaxed = mol.get_atomic_numbers()[0]*mol.get_atomic_numbers()[1]/(mol.get_distance(0, 1)/Bohr)
    e_nuc[k] = [fixed]
# bfe, bfe_el, bfe_nuc 
energies = dict()
for k in bfe.keys():
    bfe_el = bfe[k] - e_nuc[k][0]
    #bfe_elr = bfe[k] - e_nuc[k][1]
    
    energies[k] = [bfe[k], bfe_el, e_nuc[k][0]]
    
# # save data
# path = f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/{row}/energies_{bond_type}_dict'
# # uqm.save_obj(energies, path)

comps = ['CC', 'CN', 'CO', 'CF', 'NN', 'NO', 'NF', 'OO', 'OF', 'FF', '[SiH3][SiH3]', '[SiH3]P', '[SiH3]S', '[SiH3]Cl', 'PP', 'PS', 'PCl', 'SS', 'SCl', 'ClCl', '[GeH3][GeH3]','[GeH3][AsH2]','[GeH3][SeH]','[GeH3]Br','[AsH2][AsH2]','[AsH2][SeH]','[AsH2]Br','[SeH][SeH]','[SeH]Br','BrBr']
# comps = ['[SiH3][SiH3]', '[SiH3]P', '[SiH3]S', '[SiH3]Cl', 'PP', 'PS', 'PCl', 'SS', 'SCl', 'ClCl']
# comps = ['[GeH3][GeH3]','[GeH3][AsH2]','[GeH3][SeH]','[GeH3]Br','[AsH2][AsH2]','[AsH2][SeH]','[AsH2]Br','[SeH][SeH]','[SeH]Br','BrBr']
energies_ordered = []
for c in comps:
    energies_ordered.append(energies[c])
energies_ordered = np.array(energies_ordered)
np.savetxt(f'/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/distance/energies_d{dist}.txt', energies_ordered)

In [ ]:
plt.plot(energies_ordered[:,1][20:30])

In [ ]:
14**2/d

In [ ]:
atoms = ['Si', 'Cl', 'H', 'H', 'H']

In [ ]:
atoms[len(atoms)-num_hydrogens:]

In [ ]:
for f in glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_3/diatomics_single/*d2.xyz'):
    view(aio.read(f))

### Make a table with all energies

In [38]:
import pandas as pd
import pysmiles
import numpy as np
import glob
import os
import sys
import ase.io as aio

sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

In [45]:
def key2(x):
    pattern_list = ['CC', 'CN', 'CO', 'CF', 'NN', 'NO', 'NF', 'OO', 'OF', 'FF']
    for i, p in enumerate(pattern_list):
        if p in x:
            return(i)
def key3(x):
    pattern_list = ['[SiH3][SiH3]', '[SiH3]P','[SiH3]S','[SiH3]Cl','PP','PS','PCl','SS','SCl','ClCl',]
    for i, p in enumerate(pattern_list):
        if p in x:
            return(i)
def key4(x):
    pattern_list = ['[GeH3][GeH3]', '[GeH3][AsH2]', '[GeH3][SeH]', '[GeH3]Br', '[AsH2][AsH2]', '[AsH2][SeH]', '[AsH2]Br', '[SeH][SeH]', '[SeH]Br', 'BrBr']
    for i, p in enumerate(pattern_list):
        if p in x:
            return(i)

In [46]:
# get file paths

# row 2
paths2 = glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_2/diatomics_single/*_energy')
paths2.sort(key=key2)
# row 3
basepath3 = '/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_3/diatomics_single/'
frags3 = ['[SiH3]', 'P', 'S', 'Cl']
paths3 = []
for i in range(4):
    for j in range(i, 4):
        paths3.append(f'{basepath3}{frags3[i]}{frags3[j]}_energy')
paths3.sort(key=key3)
# row 4
basepath4 = '/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_4/diatomics_single/'
frags4 = ['[GeH3]', '[AsH2]', '[SeH]', 'Br']
paths4 = []
for i in range(4):
    for j in range(i, 4):
        paths4.append(f'{basepath4}{frags4[i]}{frags4[j]}_energy')
paths4.sort(key=key4)
# misxed
paths23 = glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/mixed_2_3/diatomics_single/*_energy')
paths24 = glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/mixed_2_4/diatomics_single/*_energy')
paths34 = glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/mixed_3_4/diatomics_single/*_energy')

files = paths2 + paths3 + paths4 + paths23 + paths24 + paths34

In [47]:
# read information for each compound

data = {'smiles':[], 'Z1':[], 'Z2':[], 'd':[], 'energy':[]}

for f in files:
    # get smiles
    smiles = f.split('/')[-1].split('_')[0]
    data['smiles'].append(smiles)
    # get nuclear charges
    pes = {'C':6, 'N':7, 'O':8, 'F':9, 'Si':14, 'P':15, 'S':16, 'Cl':17, 'Ge':32, 'As':33, 'Se':34, 'Br':35}
    mol = pysmiles.read_smiles(smiles)
    Z1, Z2 = pes[mol.nodes.data('element')[0]], pes[mol.nodes.data('element')[1]]
    data['Z1'].append(Z1)
    data['Z2'].append(Z2)
    # get bond length
    xyz_path = os.path.join(os.path.split(f)[0], f'{smiles}_opt.xyz')
    ase_atoms = aio.read(xyz_path)
    dist = ase_atoms.get_distance(0,1)
    data['d'].append(dist)
    # get energy
    with open(f, 'r') as fs:
        energy = float(fs.readline().strip('\n'))
    data['energy'].append(energy)

In [48]:
print(smiles)
print(Z1, Z2)
print(dist)
print(energy)

[SiH3][AsH2]
14 33
2.3665182255125545
-2527.976080967731


In [49]:
# save in dataframe
all_energies = pd.DataFrame(data)
all_energies.to_csv('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/all_energies.csv', index=False)

### Dataframe fragments

In [6]:
for f in glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/row_4/fragments_single/*_energy_relaxed'):
    with open(f, 'r') as fs:
        energy = float(fs.readline().strip('\n'))
    print(f.split('/')[-1])
    print(energy)

[GeH3][GeH3]_energy_relaxed
-2078.478952155844
[AsH2][AsH2]_energy_relaxed
-2236.7600024844014
BrBr_energy_relaxed
-2573.810129520699
[SeH][SeH]_energy_relaxed
-2401.827157663353


In [50]:
smiles = ['C', 'N', 'O', 'F'] + ['[SiH3]', 'P', 'S', 'Cl'] + ['[GeH3]', '[AsH2]', '[SeH]', 'Br']
energies = [-39.796931044615306, -55.832412853830284, -75.68331959286338, -99.67399914281202, -291.1078115883832, -342.37848151179224, -398.60997253542075, -459.9951173106298, -2078.478952155844, -2236.7600024844014, -2401.827157663353, -2573.810129520699]
pes = {'C':6, 'N':7, 'O':8, 'F':9, 'Si':14, 'P':15, 'S':16, 'Cl':17, 'Ge':32, 'As':33, 'Se':34, 'Br':35}
Z = list(pes.values())

In [51]:
# save in dataframe
all_energies = pd.DataFrame({'smiles':smiles, 'Z':Z, 'energy':energies})
all_energies.to_csv('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/all_energies_frags.csv', index=False)

### BFE

In [52]:
data = pd.read_csv('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/all_energies.csv')
data_frags = pd.read_csv('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/all_energies_frags.csv')

In [53]:
energy_frags = []
for Z1, Z2 in zip(data['Z1'], data['Z2']):
    e1 = data_frags.loc[data_frags['Z']==Z1, 'energy'].item()
    e2 = data_frags.loc[data_frags['Z']==Z2, 'energy'].item()
    energy_frags.append(e1+e2)
data['bfe'] = data['energy']-energy_frags

In [54]:
data.to_csv('/data/sahre/projects/atomic-energies/data/bonding_trends/pbe0_data/all_energies.csv', index=False)